# Linked transactions through decryption mixnet

### Message generation

The client generates a linkage param $l2 = Enc_{pk_{reg}}(l1)$.

In [1]:
from elgamal import ElGamal, ElGamalHashed
from point import Point
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from sage.all_cmdline import *
from hybrid import Hybrid
import secrets
from binascii import hexlify
import utils
from secret_sharing import SecretSharing

[4983411878214265807569785808290098840209068422821730807812853752907853544669914155652596548080570002752689808078386336024500644020431010400871335439109670310049211528477515903152413177544299891264920952591549877318088896104677071621288640589964906419114948532940645018966211566, 1444359833514147327407010573978119486186136165414776057177381542767234935595234785666995941722945949750556039989805753915172470241785894441433566414955307916280912067295957859845123650031424098125982461911420885708804313687621002583587404948635724512408657032325500736111669629592, 1198566250370932793062379703994168095186927684688753025733715078387841838123608496897800125761595768628538508356897986439072915208070056913565367270525818297301256952844203409620228276284445811865203495250618815494045955325204861619117302469691623775751441164456175530194724617199, 160637883189925524363341929950502517359879371338124656143371544632625768872824579685438588629566565609588474509172235488200098791855732806845219975703731210

In [2]:
q = Integer(8864636379641307987985582655993892125665954886639156314962949313547533029400547411406265429646812064672002946472067268869633225969806321590557478802774207323884570485379928282206644760402266682209472353933681352679372637254752717498141909964176598352222875644485687285283916382943)
Fq = GF(q)

### Generate Regulator's ElGamal keys

In [3]:
regHenc = ElGamalHashed()
regPK = regHenc.getPublicKey()
print('Regulator public key: {}'.format(regPK))

Regulator public key: (14021203869723128237559270899535048706000162930363746873947710039536957377140:18793795698473998059037471984720871290657367827598904560213236940876610214832)


### Generate Alice's parameters

Alice computes $l1$ and sends $Tx_1(l1)$ to the mixer.

In [4]:
address = secrets.randbits(160).to_bytes(20, 'little')
l1 = secrets.randbits(256)

Compute $l2 = Enc_{regPK}(l1)$

In [5]:
aliceHenc = ElGamalHashed(regPK)
l2_C1p, l2_c2 = aliceHenc.encrypt(l1)

Format data as $m = (addr || l2_{C1}.x || l2_{C1}.y || l2_{c2})$.
Note that $l2_{C1}$ is a point in the curve. The message $m$ will be sent (encrypted) to the mixnet, and the mixer will send $Tx_2(addr, l2)$.

In [6]:
l2_C1p_x = int(l2_C1p.x).to_bytes(32, 'little')
l2_C1p_y = int(l2_C1p.y).to_bytes(32, 'little')
    
m = address + l2_C1p_x + l2_C1p_y + l2_c2
print(len(address))
print(len(l2_C1p_x))
print(len(l2_C1p_y))
print(len(l2_c2))
print(len(m))

20
32
32
32
116


In [7]:
print('--address: \n   {}'.format(hexlify(address)))
print('--l1: \n   {}'.format(hexlify(l1.to_bytes(32, 'little'))))
print('--l2_C1p: \n   {}'.format(l2_C1p))
print('--l2_c2: \n   {}'.format(hexlify(l2_c2)))
print('--m: \n   {}'.format(hexlify(m)))

--address: 
   b'234a2d6bc22636b534e62c6b81012d9c4a1a0634'
--l1: 
   b'efe26f151b07cedef3663ddf0c529d06ec9670ac85cb6513096f423ed3f00b6e'
--l2_C1p: 
   (5794217405213691800798662701785814865869404085083326188114124590880902047261:14008913552901921689499484644596063959195628740367172190649919309593578843640)
--l2_c2: 
   b'8964115d127154e13d4296a0bf71bcd9c93bbc0bbc159620178abdb3f72ae471'
--m: 
   b'234a2d6bc22636b534e62c6b81012d9c4a1a06341d667384c8a059e7ceab955e08db7041240dc784698a705294fedd602c69cf0cf82d24bfd494a5db5fc6985d6c0057a7b1bee8772f2f33de9ba212269bc3f81e8964115d127154e13d4296a0bf71bcd9c93bbc0bbc159620178abdb3f72ae471'


### Compute the MAC for verification

$MAC = m \cdot k$

In [8]:
k = secrets.randbits(256)
m_int = int.from_bytes(m, 'little')
print(m_int)
print(len(m))
mac = Fq(m_int)*Fq(k)
print("MAC:\n {}".format(mac))

1009453892886402161001234709400588860846446593416317208128692594038549525270997721398086221173323533757054656054867722715370315139052224750971460782123761378619016150519938733635228996510043228075447729990363584053155395242485365326299481154425353746273273008737114911102517201443
116
MAC:
 3880312570734636430241342075586812803256755644056206001893972818119093029471221751691747644212494654589959234478801482987912445456902798610972452502048963439615421353285308067983525252905603987940669759543922735971972044216930572034935251938642721613634109087958833161306647359760


### Secret share m

In [11]:
ss_m = SecretSharing(Fq(m_int), 5)
ss_k = SecretSharing(Fq(k), 5)
m_shared = ss_m.share()
k_shared = ss_k.share()
print("\nShares of m:\n")
for i in range(0, len(m_shared)): print("   m({}): {}".format(i, m_shared[i]))
print("\nShares of k:\n")
for i in range(0, len(k_shared)): print("   k({}): {}".format(i, k_shared[i]))


Shares of m:

   m(0): 851214546297781002539876611820484179295885362092936739240448606188375573060499473420273463028320956561743321686467647536520474278502390854857865029936060919506910086314008046838653708650813229804603031748483348788413816479606190388796444242112221427467570418418734563550078329645
   m(1): 1559295608084623423771046941362831945561111036285206599742807312625391613086156356213137573028992725877564303772050879439158727087565808925733380087178403713478655364970784067640540384958626876614431818209610583262338510688661630686096291350695462067545299485062986506261643139303
   m(2): 1261260236782657333277740622818235587353364185198684732381702976736077226833969264062159276685305500556375264006539698210149812271507484503833727328038985587123702200127439075974527302007517841138008773909217902074752318065132877975536285271250395062359465037594001783979194014363
   m(3): 12701596151030529834677574170961672859619478161387262686666711376001251098632302943195056775647344846650146